In [29]:
import gmsh
import meshio
import numpy
import pygmsh
import h5py
import matplotlib.pyplot as plt
import numpy as np
import subprocess

In [14]:
resolution = 0.01
LT = 10
LB = 10
W = 5

point0 = (0, 0, 0)
point1 = (LB, 0, 0)
point2 = (LT, W, 0)
point3 = (0, W, 0)
# c = (2.5, 2.5, 0)
centers = [
           (1, 1, 0),
           (2, 2.75, 0),
#            (1, 4.5, 0),
           (3, 1, 0),
           (4, 2.75, 0),
#            (3, 4.5, 0),
           (5, 1, 0),
           (6, 2.75, 0),
#            (5, 4.5, 0),
           (7, 1, 0),
           (8, 2.75, 0),
           (7, 4.25, 0),
]
r = 0.75

In [15]:
geometry = pygmsh.geo.Geometry()
model = geometry.__enter__()

In [16]:
points = [model.add_point(point0, mesh_size=resolution),
          model.add_point(point1, mesh_size=resolution),
          model.add_point(point2, mesh_size=resolution),
          model.add_point(point3, mesh_size=resolution),
          ]

In [17]:
channel_lines = [model.add_line(points[i], points[i+1])
  for i in range(-1, len(points)-1)]
channel_loop = model.add_curve_loop(channel_lines)
circles = [model.add_circle(c, r, mesh_size=resolution) for c in centers]
holes = [circle.curve_loop for circle in circles]
plane_surface = model.add_plane_surface(channel_loop, holes=holes)
model.synchronize()

In [18]:
volume_marker = 6
model.add_physical([plane_surface], "Volume")
model.add_physical([channel_lines[0]], "Left")
model.add_physical([channel_lines[2]], "Right")
model.add_physical([channel_lines[1], channel_lines[3]], "Insulated")
for i, c in enumerate(circles):
    model.add_physical(c.curve_loop.curves, "void%d" % i)

In [19]:
geometry.generate_mesh(dim=2)
gmsh.write("mesh.msh")
gmsh.clear()
geometry.__exit__()

In [20]:
mesh_from_file = meshio.read("mesh.msh")

In [21]:
def create_mesh(mesh, cell_type, prune_z=False):
    cells = mesh.get_cells_type(cell_type)
    cell_data = mesh.get_cell_data("gmsh:physical", cell_type)
    points = mesh.points[:,:2] if prune_z else mesh.points
    out_mesh = meshio.Mesh(points=points, cells={cell_type: cells}, cell_data={"name_to_read":[cell_data]})
    return out_mesh

In [22]:
line_mesh = create_mesh(mesh_from_file, "line", prune_z=True)
meshio.write("facet_mesh.xdmf", line_mesh)

triangle_mesh = create_mesh(mesh_from_file, "triangle", prune_z=True)
meshio.write("mesh.xdmf", triangle_mesh)


In [28]:
subprocess.Popen(["mpirun", "-n", "2", "python3 demo_poisson.py"])

In [63]:
# load simulation results
with h5py.File("poisson.h5", 'r') as hf:
    values = np.asarray(hf.get("Function").get("f_4").get("0"))
    positions = np.asarray(hf.get("Mesh").get("Grid").get("geometry"))

results = np.hstack((positions, values))

In [64]:
idx = np.lexsort((results[:, 0], results[:, 1]))
results = results[idx]

In [ ]:
def grad(data_arr):
    """
    indep_var -> array of y-values
    dep_var -> array of x-values for which y-values are provided
    """
    gradient = np.nan(data_arr.shape)
    for row in np.nditer(data_arr):
        print(row)
        break

In [90]:
x_series = np.arange(0, 10.1, 0.1)
y_series = np.arange(0, 5.1, 0.1)
for y in y_series:
    inderesults[np.where(np.isclose(results[:, 1], y)), :])
    break

[[[ 0.00000000e+00  0.00000000e+00  1.00000000e+00]
  [ 1.00000000e-02  0.00000000e+00  9.99465508e-01]
  [ 2.00000000e-02  0.00000000e+00  9.98930813e-01]
  ...
  [ 9.98000000e+00  0.00000000e+00  1.13777939e-03]
  [ 9.99000000e+00  0.00000000e+00  5.68887872e-04]
  [ 1.00000000e+01  0.00000000e+00 -1.26318483e-19]]]


In [ ]:
grid_size = int(grid_id.split('_')[0])
grid_data = np.empty([grid_size+1, grid_size+1, grid_size+1])
grid_data[:] = 0

for _, row in enumerate(results):
    coords = [int(v) for v in row[:3]]
    i_x, i_y, i_z = coords
    value = row[3]
    grid_data[i_x, i_y, i_z] = value

[u_x, _, _] = np.gradient(grid_data)
delta_phi_dx = -1 / grid_size
kappa_eff = np.average(u_x) / delta_phi_dx